In [13]:
import glob
import pickle
import numpy
import pandas as pd
from music21 import converter, instrument, note, stream, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


rectified linear unit (ReLU)

In [ ]:
#song_location = glob.glob('midi/*.mid')
#song_location[:5]

['doug_mckenzie_midi/Chelsea Bridge.mid',
 "doug_mckenzie_midi/I'm Confessin' - solo.mid",
 'doug_mckenzie_midi/Very Early solo piano.mid',
 "doug_mckenzie_midi/Goin'_Home_(Real_Time)_(Antonin_Dvorak).mid",
 'doug_mckenzie_midi/dearlybeloved.mid']

In [ ]:
def get_notes():
    """ Get all the notes and chords from the midi files in the ./midi_songs directory """
    notes = []

    for file in glob.glob("/content/drive/MyDrive/Colab Notebooks/doug_mckenzie_midi_5_songs/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('/content/drive/MyDrive/Colab Notebooks/Skuldur Model doug data 5 songs/data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [ ]:
get_notes()

Parsing /content/drive/MyDrive/Colab Notebooks/doug_mckenzie_midi_5_songs/accustomed.mid
Parsing /content/drive/MyDrive/Colab Notebooks/doug_mckenzie_midi_5_songs/afine-2.mid
Parsing /content/drive/MyDrive/Colab Notebooks/doug_mckenzie_midi_5_songs/AfterYou.mid
Parsing /content/drive/MyDrive/Colab Notebooks/doug_mckenzie_midi_5_songs/A Sleepin' Bee.mid


['E-4',
 'E-3',
 '5',
 'G4',
 'G3',
 'B-4',
 '2.7',
 'E-2',
 'B-2',
 '7.10',
 'G3',
 'D4',
 'B-3',
 'D4',
 'B-4',
 'G4',
 'D4',
 '7.10',
 '7.10',
 '10.2',
 'F4',
 '10.2',
 'F#3',
 'G#2',
 'E-3',
 'F5',
 'F5',
 '10.2.5',
 'F#3',
 'F6',
 'G6',
 'C4',
 'F6',
 '10.2',
 '2.5',
 'F4',
 'F6',
 'C4',
 'F6',
 'F#3',
 'F4',
 '10.2',
 'F4',
 'G2',
 'A3',
 'A3',
 'B-3',
 'D4',
 'F4',
 'A4',
 'C5',
 'D5',
 'G4',
 'E-4',
 'B3',
 'C3',
 'E-5',
 'E-4',
 'B-3',
 'G4',
 'C3',
 '11.3',
 'B3',
 'G4',
 '10.2',
 '7.0',
 'E-4',
 'G3',
 'C3',
 '3.5',
 'G#3',
 'B-4',
 'C3',
 'E-4',
 'G3',
 'G#3',
 'G#3',
 'B-4',
 'E-4',
 'B-3',
 'B-4',
 'E-4',
 'G#3',
 'G#4',
 'C#4',
 'A3',
 'F4',
 'B2',
 'E-3',
 'G#4',
 'C#4',
 'F4',
 'A3',
 'G#4',
 'G#4',
 '0.3',
 'B-2',
 'G#3',
 'B-2',
 'E-4',
 'C4',
 'B-2',
 '2.4.7',
 '8.11',
 'G5',
 '8.11.2',
 '2.4',
 'G6',
 '8.11',
 '2.4',
 'B6',
 '4.7.8',
 'D7',
 'G4',
 'D5',
 '7.0',
 'A2',
 'D5',
 'C4',
 'D5',
 'E4',
 'D5',
 'C5',
 'C4',
 'E-4',
 'F#4',
 'D3',
 'F#3',
 '0.3.6.8',
 'A4'

In [ ]:
notes = pd.read_pickle(r'/content/drive/MyDrive/Colab Notebooks/Skuldur Model doug data 5 songs/data/notes')
notes[:10]

['E-4', 'E-3', '5', 'G4', 'G3', 'B-4', '2.7', 'E-2', 'B-2', '7.10']

In [ ]:
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = np_utils.to_categorical(network_output)

    return (network_input, network_output)

In [ ]:
# get amount of pitch names
n_vocab = len(set(notes))

In [ ]:
prepare_sequences(notes, n_vocab)

(array([[[0.89275362],
         [0.88985507],
         [0.47246377],
         ...,
         [0.96811594],
         [0.7942029 ],
         [0.89275362]],
 
        [[0.88985507],
         [0.47246377],
         [0.9884058 ],
         ...,
         [0.7942029 ],
         [0.89275362],
         [0.85217391]],
 
        [[0.47246377],
         [0.9884058 ],
         [0.98550725],
         ...,
         [0.89275362],
         [0.85217391],
         [0.7942029 ]],
 
        ...,
 
        [[0.96811594],
         [0.31304348],
         [0.31304348],
         ...,
         [0.72463768],
         [0.67826087],
         [0.39710145]],
 
        [[0.31304348],
         [0.31304348],
         [0.33623188],
         ...,
         [0.67826087],
         [0.39710145],
         [0.63768116]],
 
        [[0.31304348],
         [0.33623188],
         [0.45507246],
         ...,
         [0.39710145],
         [0.63768116],
         [0.96231884]]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 

In [ ]:
(network_input,network_output) = prepare_sequences(notes, n_vocab)
network_input

array([[[0.89275362],
        [0.88985507],
        [0.47246377],
        ...,
        [0.96811594],
        [0.7942029 ],
        [0.89275362]],

       [[0.88985507],
        [0.47246377],
        [0.9884058 ],
        ...,
        [0.7942029 ],
        [0.89275362],
        [0.85217391]],

       [[0.47246377],
        [0.9884058 ],
        [0.98550725],
        ...,
        [0.89275362],
        [0.85217391],
        [0.7942029 ]],

       ...,

       [[0.96811594],
        [0.31304348],
        [0.31304348],
        ...,
        [0.72463768],
        [0.67826087],
        [0.39710145]],

       [[0.31304348],
        [0.31304348],
        [0.33623188],
        ...,
        [0.67826087],
        [0.39710145],
        [0.63768116]],

       [[0.31304348],
        [0.33623188],
        [0.45507246],
        ...,
        [0.39710145],
        [0.63768116],
        [0.96231884]]])

In [ ]:
network_output

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
# def create_network(network_input, n_vocab):
#     """ create the structure of the neural network """
#     model = Sequential()
#     model.add(LSTM(
#         128,
#         input_shape=(network_input.shape[1], network_input.shape[2]),
#         recurrent_dropout=0.3,
#         return_sequences=True
#     ))
#     model.add(LSTM(128, return_sequences=True, recurrent_dropout=0.3,))
#     model.add(LSTM(128))
#     model.add(BatchNorm())
#     model.add(Dropout(0.3))
#     model.add(Dense(256))
#     model.add(Activation('relu'))
#     model.add(BatchNorm())
#     model.add(Dropout(0.3))
#     model.add(Dense(n_vocab))
#     model.add(Activation('softmax'))
#     model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

#     return model

In [ ]:
# model = create_network(network_input, n_vocab)
# model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 100, 128)          66560     
                                                                 
 lstm_3 (LSTM)               (None, 100, 128)          131584    
                                                                 
 lstm_4 (LSTM)               (None, 128)               131584    
                                                                 
 batch_normalization_2 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               33024     
                                                      

In [ ]:
def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(LSTM(
        128,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.2,
        return_sequences=True
    ))
    model.add(LSTM(128, return_sequences=False))
    # model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256, activation='relu'))
    # model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

In [ ]:
model = create_network(network_input, n_vocab)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 128)          66560     
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 345)               88665     
                                                                 
Total params: 319,833
Trainable params: 319,833
Non-trai

In [ ]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "/content/drive/MyDrive/Colab Notebooks/Skuldur Model doug data 5 songs/weights/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    # model.fit(network_input, network_output, epochs=10, batch_size=128, callbacks=callbacks_list)
    model.fit(network_input, network_output, epochs=200, batch_size=32, callbacks=callbacks_list)

In [ ]:
train(model, network_input, network_output)

Epoch 1/200
129/129 [==============================] - 44s 339ms/step - loss: 4.8497
Epoch 2/200
129/129 [==============================] - 44s 341ms/step - loss: 4.8465
Epoch 3/200
129/129 [==============================] - 43s 337ms/step - loss: 4.8333
Epoch 4/200
129/129 [==============================] - 43s 335ms/step - loss: 4.8325
Epoch 5/200
129/129 [==============================] - 44s 338ms/step - loss: 4.8261
Epoch 6/200
129/129 [==============================] - 44s 339ms/step - loss: 4.8117
Epoch 7/200
129/129 [==============================] - 44s 339ms/step - loss: 4.8198
Epoch 8/200
129/129 [==============================] - 44s 338ms/step - loss: 4.7940
Epoch 9/200
129/129 [==============================] - 44s 341ms/step - loss: 4.8103
Epoch 10/200
129/129 [==============================] - 43s 337ms/step - loss: 4.7666
Epoch 11/200
129/129 [==============================] - 44s 339ms/step - loss: 4.7571
Epoch 12/200
129/129 [==============================] - 44s 338

In [ ]:
# def train_network():
#     """ Train a Neural Network to generate music """
#     notes = get_notes()

#     # get amount of pitch names
#     n_vocab = len(set(notes))

#     network_input, network_output = prepare_sequences(notes, n_vocab)

#     model = create_network(network_input, n_vocab)

#     train(model, network_input, network_output)

In [14]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('/content/drive/MyDrive/Colab Notebooks/Skuldur Model doug data 5 songs/data/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences(notes, pitchnames, n_vocab)
    model = create_network(normalized_input, n_vocab)
    # Load the weights to each node
    model.load_weights('/content/drive/MyDrive/Colab Notebooks/Skuldur Model doug data 5 songs/weights/weights-improvement-01-4.8497-bigger.hdf5')
    # model.load_weights('/content/drive/MyDrive/Colab Notebooks/Skuldur Model doug data 5 songs/weights/weights-improvement-200-2.3949-bigger.hdf5')
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [ ]:
def prepare_sequences(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 100
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [ ]:
# def create_network(network_input, n_vocab):
#     """ create the structure of the neural network """
#     model = Sequential()
#     model.add(LSTM(
#         128,
#         input_shape=(network_input.shape[1], network_input.shape[2]),
#         recurrent_dropout=0.3,
#         return_sequences=True
#     ))
#     model.add(LSTM(128, return_sequences=True, recurrent_dropout=0.3,))
#     model.add(LSTM(128))
#     model.add(BatchNorm())
#     model.add(Dropout(0.3))
#     model.add(Dense(256))
#     model.add(Activation('relu'))
#     model.add(BatchNorm())
#     model.add(Dropout(0.3))
#     model.add(Dense(n_vocab))
#     model.add(Activation('softmax'))
#     model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

#     # Load the weights to each node
#     model.load_weights('/content/drive/MyDrive/Colab Notebooks/Skuldur Model doug data/weights/weights-improvement-03-4.7436-bigger.hdf5')

#     return model

In [ ]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = numpy.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [15]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='/content/drive/MyDrive/Colab Notebooks/Skuldur Model doug data 5 songs/test_output_epoch1.mid')
    # midi_stream.write('midi', fp='/content/drive/MyDrive/Colab Notebooks/Skuldur Model doug data 5 songs/test_output_epoch200.mid')

In [16]:
generate()

In [ ]:
!pip install pygame

     |████████████████████████████████| 18.3 MB 95 kB/s 


In [ ]:
import pygame

pygame 2.1.0 (SDL 2.0.16, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
def play_music(music_file):
    """
    stream music with mixer.music module in blocking manner
    this will stream the sound from disk while playing
    """
    clock = pygame.time.Clock()
    try:
        pygame.mixer.music.load(music_file)
        print ("Music file %s loaded!" % music_file)
    except pygame.error:
        print ("File %s not found! (%s)" % (music_file, pygame.get_error()))
        return
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():
        # check if playback has finished
        clock.tick(30)
# pick a midi music file you have ...
# (if not in working folder use full path)

In [ ]:
def play_midi(midi_file):
    freq = 44100    # audio CD quality
    bitsize = -16   # unsigned 16 bit
    channels = 2    # 1 is mono, 2 is stereo
    buffer = 1024    # number of samples
    pygame.mixer.init(freq, bitsize, channels, buffer)

    # optional volume 0 to 1.0
    pygame.mixer.music.set_volume(0.8)
    try:
        play_music(midi_file)
    except KeyboardInterrupt:
        # if user hits Ctrl/C then exit
        # (works only in console mode)
        pygame.mixer.music.fadeout(1000)
        pygame.mixer.music.stop()
        raise SystemExit

In [ ]:
### Play the Jazz music
play_midi('/content/drive/MyDrive/Colab Notebooks/test_output.mid')

error: ignored